# Attractor municipalities

In this script we select the attractor municipalitites based on the complexity index.

We cluster the data using in one axis the complexity of the X municiaplity and in the other the number of municipalitites that have a complexity index at 1/2 standard deviations from the complexity index of the X municipality.

We make 7 clusters and select the municipalities that fall in the last part of the distribution.

The output of this script is a csv file with the attractors.

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import math

import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from shapely.geometry import Polygon
from sklearn.cluster import DBSCAN

# Paths and names

In [8]:
# We define the paths and names for the input files

entities_path = 'data/marco_geoestadistico_2018/'

# complexity_path = 'ECI_2014__municipal_A111A_H001A_6.csv'

# We define the paths and names for the output files

In [3]:
# We define a dictionary to read the shape files
entities= {
    '01' :['01Ags','01_aguascalientes'],
    '02' :['02BC','02_bajacalifornia'],
    '03' :['03BCS','03_bajacaliforniasur'],
    '04' :['04Camp', '04_campeche'],
    '05' :['05Coah','05_coahuiladezaragoza'],
    '06' :['06Col','06_colima'],
    '07' :[ '07Chs','07_chiapas'],
    '08' :['08Chih','08_chihuahua'],
    '09' :['09CdMx','09_ciudaddemexico'],
    '10' :['10Dur','10_durango'],
    '11' :['11Gto', '11_guanajuato'],
    '12' :['12Gro','12_guerrero'],
    '13' :['13Hgo','13_hidalgo'],
    '14' :['14Jal','14_jalisco'],
    '15' :['15EdoMex','15_mexico'],
    '16' :['16Mich','16_michoacandeocampo'],
    '17' :['17Mor','17_morelos'],
    '18' :['18Nay','18_nayarit'],
    '19' :['19NL', '19_nuevoleon'],
    '20' :['20Oax', '20_oaxaca'],
    '21' :['21Pue', '21_puebla'],
    '22' :['22Qro','22_queretaro'],
    '23' :['23QRoo','23_quintanaroo'], 
    '24' :['24SLP','24_sanluispotosi'],
    '25' :['25Sin','25_sinaloa'],
    '26' :['26Son','26_sonora'],
    '27' :['27Tab','27_tabasco'],
    '28' :['28Tamps','28_tamaulipas'],
    '29' :['29Tlax','29_tlaxcala'],
    '30' :['30Ver','30_veracruzignaciodelallave'],
    '31' :[ '31Yuc','31_yucatan'],
    '32' :[ '32Zac','32_zacatecas']

    
}

In [4]:
complexity = pd.read_csv('/home/lorena/Documents/Economia/clusterizacion_y_complejidad/Data/Municipality/Produccion_bruta_sobre_empleo/'+complexity_path)

complexity['clave_entidad'] = complexity.clave_entidad.apply(lambda l:str(l).zfill(5))
print("complexity data: ",complexity.shape)
complexity.head(1)

NameError: name 'complexity_path' is not defined

# Entities shapefiles

In [9]:
urban_shapes = []

for i in range(1,33):
    a = str(i).zfill(2)
    
    urban_agebs = gpd.read_file(entities_path + entities[a][1]+'/conjunto de datos/'+entities[a][1][0:2]+'mun'+'.shp')
    
    urban_shapes.append(urban_agebs)
    print(a, urban_agebs.shape)
    
    
urban_shapes = pd.concat(urban_shapes)

display(urban_shapes.head(1))

01 (11, 5)
02 (5, 5)
03 (5, 5)
04 (11, 5)
05 (38, 5)
06 (10, 5)
07 (123, 5)
08 (67, 5)
09 (16, 5)
10 (39, 5)
11 (46, 5)
12 (81, 5)
13 (84, 5)
14 (125, 5)
15 (125, 5)
16 (113, 5)
17 (33, 5)
18 (20, 5)
19 (51, 5)
20 (570, 5)
21 (217, 5)
22 (18, 5)
23 (11, 5)
24 (58, 5)
25 (18, 5)
26 (72, 5)
27 (17, 5)
28 (43, 5)
29 (60, 5)
30 (212, 5)
31 (106, 5)
32 (58, 5)


,CVEGEO,CVE_ENT,CVE_MUN,NOMGEO,geometry
0,01001,01,001,Aguascalientes,"POLYGON ((2489072.503 1115771.584, 2489352.774..."


In [10]:
gs = gpd.GeoSeries(urban_shapes['geometry'])

In [11]:
gs.crs = {'init' :'epsg:4326'}

In [12]:
area = gs.area

In [13]:
urban_shapes['area'] = area

In [14]:
centroid = gs.centroid
urban_shapes['centroid'] = centroid

In [15]:
urban_shapes

,CVEGEO,CVE_ENT,CVE_MUN,NOMGEO,geometry,area,centroid
0,01001,01,001,Aguascalientes,"POLYGON ((2489072.503 1115771.584, 2489352.774...",1.166359e+09,POINT (2469560.280 1088371.808)
1,01002,01,002,Asientos,"POLYGON ((2494680.261 1141224.505, 2494749.948...",5.433743e+08,POINT (2495320.670 1123052.741)
2,01003,01,003,Calvillo,"POLYGON ((2429607.454 1120262.254, 2429071.902...",9.233578e+08,POINT (2427526.305 1098350.349)
3,01004,01,004,CosÃ­o,"POLYGON ((2470517.824 1155028.588, 2470552.248...",1.283835e+08,POINT (2469565.650 1148876.041)
4,01005,01,005,JesÃºs MarÃ­a,"POLYGON ((2465526.729 1114740.466, 2465752.546...",4.998951e+08,POINT (2454187.093 1101710.704)
...,...,...,...,...,...,...,...
53,32054,32,054,Villa Hidalgo,"POLYGON ((2531223.368 1166188.348, 2531242.386...",3.714260e+08,POINT (2529259.994 1154020.813)
54,32055,32,055,Villanueva,"POLYGON ((2420011.778 1183552.394, 2420043.461...",2.156506e+09,POINT (2412708.057 1142220.219)
55,32056,32,056,Zacatecas,"POLYGON ((2444891.234 1194945.507, 2444892.428...",4.367857e+08,POINT (2430721.277 1190326.977)
56,32057,32,057,Trancoso,"POLYGON ((2476976.320 1198274.003, 2476566.139...",2.184470e+08,POINT (2468126.403 1192389.503)


In [16]:
anchors = pd.read_csv('complejidad_alta.csv')

In [17]:
anchors

,cve_mun,NOMGEO
0,14039,Guadalajara
1,9016,Miguel Hidalgo
2,19039,Monterrey
3,9015,Cuauhtémoc
4,24028,San Luis Potosí
...,...,...
132,5025,Piedras Negras
133,7078,San Cristóbal de las Casas
134,21140,San Pedro Cholula
135,29005,Apizaco


In [18]:
for i in range(0,137):
    anchors['cve_mun'].iloc[i] = str(anchors['cve_mun'].iloc[i]).zfill(5)

/Users/ruijiehuang/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [19]:
anchors_shapes = urban_shapes.loc[urban_shapes['CVEGEO'].isin(anchors['cve_mun'])]

In [20]:
anchors_shapes['area'].describe()

count    1.370000e+02
mean     1.960480e+09
std      5.306802e+09
min      2.296278e+07
25%      1.133226e+08
50%      4.732433e+08
75%      1.447677e+09
max      5.323784e+10
Name: area, dtype: float64

In [21]:
min_area = anchors_shapes['area'].min()

In [22]:
radius = []
for i in range(0,137):
    area = anchors_shapes['area'].iloc[i]
    a = (area * 50)/min_area
    d = a - 50
    radius.append(math.log(d,10) + 50)

In [24]:
radius

[53.396142295206786,
 53.01640219450073,
 54.535818297960425,
 53.427691098591716,
 55.063978523004266,
 54.53234556839158,
 53.904730608980664,
 53.8431873285256,
 54.26847546496616,
 53.424023284107044,
 52.991428577800534,
 54.1620413305353,
 54.08140366062332,
 53.433770621998235,
 53.19484489006783,
 53.46070190436241,
 53.320007581249655,
 52.909017665511655,
 53.323896055164546,
 52.833337822339544,
 53.891298096565734,
 54.259355448178304,
 53.04360339858398,
 53.88874963082257,
 51.35331806974077,
 51.82451093782506,
 52.02016576823021,
 52.14693597820257,
 35.85159020379288,
 52.29008513647473,
 52.19765727786573,
 52.13327268849387,
 52.80017851873038,
 52.29479320208088,
 50.89257250011823,
 51.3097990477361,
 51.7026665227573,
 51.36746173263323,
 54.29994227427863,
 53.24723487879057,
 53.05858252057692,
 53.25223531040336,
 53.41256751853132,
 53.199087761602975,
 52.93855541133291,
 53.0461810050154,
 53.57055075227917,
 53.67210661483749,
 52.45190084753797,
 52.064863

In [72]:
gs_anchors = gpd.GeoSeries(anchors_shapes['geometry'])

gs_anchors.crs = {'init' :'epsg:4326'}
area_anchors = gs_anchors.area
anchors_shapes['area'] = area_anchors

anchors_shapes['radius'] = radius


/Users/ruijiehuang/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/ruijiehuang/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [73]:
anchors_shapes

,CVEGEO,CVE_ENT,CVE_MUN,NOMGEO,geometry,area,centroid,radius
0,01001,01,001,Aguascalientes,"POLYGON ((2489072.503 1115771.584, 2489352.774...",1.166359e+09,POINT (2469560.280 1088371.808),53.396142
4,01005,01,005,JesÃºs MarÃ­a,"POLYGON ((2465526.729 1114740.466, 2465752.546...",4.998951e+08,POINT (2454187.093 1101710.704),53.016402
0,02002,02,002,Mexicali,"MULTIPOLYGON (((1437920.653 1886029.010, 14379...",1.579446e+10,POINT (1248372.005 2243071.923),54.535818
2,02004,02,004,Tijuana,"MULTIPOLYGON (((1058853.778 2337446.069, 10588...",1.252511e+09,POINT (1098843.969 2332138.009),53.427691
4,02001,02,001,Ensenada,"MULTIPOLYGON (((1187249.508 2308085.809, 11871...",5.323784e+10,POINT (1252048.714 2052135.533),55.063979
...,...,...,...,...,...,...,...,...
189,30193,30,193,Veracruz,"MULTIPOLYGON (((3119599.686 810246.093, 311958...",2.458809e+08,POINT (3105985.529 811025.028),52.686091
49,31050,31,050,MÃ©rida,"POLYGON ((3786444.915 1052263.540, 3786467.873...",8.755765e+08,POINT (3777941.090 1052260.898),53.268698
9,32010,32,010,Fresnillo,"POLYGON ((2407833.677 1282843.912, 2408016.079...",5.037308e+09,POINT (2398614.078 1245714.261),54.038160
16,32017,32,017,Guadalupe,"POLYGON ((2444817.153 1194648.295, 2444821.884...",8.083220e+08,POINT (2458349.055 1195113.070),53.233014


In [89]:
polygon_list = []

def find_circle(point, radius_individual):
    global polygon_list
    
    bufferLength = radius_individual*1000  
    polygonSides = 360
    
    x = point.x
    y = point.y

    angles = np.linspace(0, 2 * np.pi, polygonSides, endpoint=False)
    points_list = [(x + np.sin(a) * bufferLength,
                    y + np.cos(a) * bufferLength)
                   for a in angles]
    polygon_list.append(Polygon(points_list))



In [94]:
for i in range(anchors_shapes.shape[0]):

    find_circle(anchors_shapes['centroid'].iloc[i],anchors_shapes['radius'].iloc[i])

    
    

In [92]:
anchors_shapes['circlerange'] = polygon_list

/Users/ruijiehuang/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [95]:
anchors_shapes

,CVEGEO,CVE_ENT,CVE_MUN,NOMGEO,geometry,area,centroid,radius,circlerange
0,01001,01,001,Aguascalientes,"POLYGON ((2489072.503 1115771.584, 2489352.774...",1.166359e+09,POINT (2469560.280 1088371.808),53.396142,"POLYGON ((2469560.279617607 1141767.949829568,..."
4,01005,01,005,JesÃºs MarÃ­a,"POLYGON ((2465526.729 1114740.466, 2465752.546...",4.998951e+08,POINT (2454187.093 1101710.704),53.016402,"POLYGON ((2454187.093354649 1154727.106215929,..."
0,02002,02,002,Mexicali,"MULTIPOLYGON (((1437920.653 1886029.010, 14379...",1.579446e+10,POINT (1248372.005 2243071.923),54.535818,"POLYGON ((1248372.005337266 2297607.741569893,..."
2,02004,02,004,Tijuana,"MULTIPOLYGON (((1058853.778 2337446.069, 10588...",1.252511e+09,POINT (1098843.969 2332138.009),53.427691,"POLYGON ((1098843.969359343 2385565.700558116,..."
4,02001,02,001,Ensenada,"MULTIPOLYGON (((1187249.508 2308085.809, 11871...",5.323784e+10,POINT (1252048.714 2052135.533),55.063979,"POLYGON ((1252048.714176597 2107199.511988183,..."
...,...,...,...,...,...,...,...,...,...
189,30193,30,193,Veracruz,"MULTIPOLYGON (((3119599.686 810246.093, 311958...",2.458809e+08,POINT (3105985.529 811025.028),52.686091,"POLYGON ((3105985.529475033 863711.1191755843,..."
49,31050,31,050,MÃ©rida,"POLYGON ((3786444.915 1052263.540, 3786467.873...",8.755765e+08,POINT (3777941.090 1052260.898),53.268698,"POLYGON ((3777941.090165559 1105529.596144433,..."
9,32010,32,010,Fresnillo,"POLYGON ((2407833.677 1282843.912, 2408016.079...",5.037308e+09,POINT (2398614.078 1245714.261),54.038160,"POLYGON ((2398614.078271838 1299752.420510422,..."
16,32017,32,017,Guadalupe,"POLYGON ((2444817.153 1194648.295, 2444821.884...",8.083220e+08,POINT (2458349.055 1195113.070),53.233014,"POLYGON ((2458349.05531734 1248346.084306987, ..."


In [99]:
dummy = urban_shapes[['CVEGEO','CVE_ENT','CVE_MUN', 'centroid']]

dummy

KeyError: "['CVEGEO', 'CVE_MUN', 'CVE_ENT'] not in index"

# Merge shapes y complejidad

In [23]:
# We rename the columns in the shapefiles and drop the columns we don't need 
urban_shapes = urban_shapes.rename(columns={'CVEGEO':'clave_entidad'})
del urban_shapes['CVE_ENT']
del urban_shapes['CVE_MUN']

print("Shapefile: ",urban_shapes.shape)
urban_shapes.head(1)

# We merge the polygons of the entities with the complexity index for each entity
mexico = urban_shapes.merge(complexity, on='clave_entidad' , how='outer')

# There are a few municipalities for which we don't have the complexity index because they're new in the country
# we take those out, they're 7
mexico = mexico[~mexico.ECI.isnull()]
mexico.plot()


Shapefile:  (2463, 5)


NameError: name 'complexity' is not defined

# Selección de focos atractores

In [ ]:
# Desviación estándar de los datos de complejidad
sd = np.std(mexico.ECI.values.tolist())

# Le asignamos a cada municipio el número de municipios cuya complejidad cae a +/- 1/2 de desviación estándar 
# de su complejidad.

final = []
for i in range(len(mexico.ECI.values.tolist())):
    
    d = mexico.ECI.values.tolist()[i]
    m = mexico.cve_mun.values.tolist()[i]
    
    mas_sd = d + (sd/2)
    menos_sd = d - (sd/2)
   
    cluster = 0
    for j in mexico.ECI.values.tolist():
        if j > menos_sd and j < mas_sd:
            cluster = cluster +1
           
            
    a = (m,d, cluster)
    
    final.append(a)
   

df = pd.DataFrame(final)
df = df.rename(columns={1:'ECI',2:'normalidad',0:'cve_mun'})
df = df.sort_values(by='ECI', ascending=False)

# Normalizamos los datos de complejidad y "normalidad" para clusterizar
df['ECI_norm'] = (df['ECI'] -df['ECI'].min()) / (df['ECI'].max()- df['ECI'].min())
df['normalidad_norm'] = (df['normalidad'] - df['normalidad'].min()) / (df['normalidad'].max()-df['normalidad'].min())

#### Usamos los datos de complejidad y "normalidad" normalizados para clusterizar al país

In [ ]:
X = df[['ECI_norm','normalidad_norm']].values

# Dividimos al país en 7 clusters
kmeans = KMeans(n_clusters=7, random_state=0).fit(X)

df['cluster'] = kmeans.labels_.tolist()

plt.figure(figsize=(15,15))
for i in range(0,7):
    sub = df[df.cluster == i]
    
    plt.scatter(x=sub['ECI'], y=sub['normalidad'])
   
    print(i, sub['ECI'].max(), sub['ECI'].min(), sub['normalidad'].max(), sub['normalidad'].min(), sub.shape[0])
    

In [ ]:
maximos = df[df.cluster == 5][['cve_mun']]

In [ ]:
nombres = urban_shapes[['NOMGEO', 'cve_mun']]

In [ ]:
maximos = maximos.merge(nombres, on='cve_mun', how='left')

In [ ]:
maximos.to_csv('complejidad_alta.csv',index=False, encoding='latin')

In [ ]:
maximos

In [ ]:
#complejidad.tail(50).to_csv('municipios_menos_complejos.csv', index=False, )

# Proximidad con Aeroespacial

In [ ]:
acts= [311511,541930,336410,711320,541820,561590,336340,339992,335920,
339991,327213,236212,623221,331220,311940,331210,335920,431193,711320,
541930,519110,333242,333412,331220,339113,334519,541370,339113,'6232CC',
541870,493120,336610,623221,312131]


In [ ]:
for i in range(len(acts)):
    acts[i] = str(acts[i])

In [ ]:
relate = pd.read_csv('/home/lorena/Documents/Economia/clusterizacion_y_complejidad/Data/Municipality/Produccion_bruta_sobre_empleo/relatedness_matrix_clave_actividad_economica_and_cve_mun_2014_A111A_H001A_6.csv')
print(relate.shape)
display(relate.head())

In [ ]:
relate = relate[relate.clave_actividad.isin(acts)]

In [ ]:
aero = []
peso = []
for mun in relate.set_index('clave_actividad').columns:
    print(mun)
    #m = 0
    for value in relate[mun].values:
        m=0
        if value > 0.2:
            m = m +1


    if m != 0:
        aero.append(str(mun).zfill(5))
        peso.append(m)
    #break

In [ ]:
data= pd.DataFrame({'clave_entidad':aero, 'peso':peso})
print(data.shape)

In [ ]:
data['peso'] = data['peso']  / data['peso'].max() + 1

In [ ]:
data.max()

In [ ]:
urban_shapes_1 = urban_shapes.merge(data, on='clave_entidad', how='left')
urban_shapes_1.loc[urban_shapes_1.peso.isnull(),'peso'] = 0.5

In [ ]:
urban_shapes_1['peso'] = (urban_shapes_1['peso'] - urban_shapes_1['peso'].min()) / (urban_shapes_1['peso'].max() - urban_shapes_1['peso'].min())

In [ ]:
urban_shapes_1.plot(figsize=(15,15),column='peso', cmap = 'Greens', vmax=2, vmin=0)

In [ ]:
x = urban_shapes_1[urban_shapes_1.peso != 0.5][['clave_entidad','NOMGEO','peso']]
x.to_csv('municipios_aeroespacial.csv',index=False, encoding='latin')